In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("berkanoztas/synthetic-transaction-monitoring-dataset-aml")

print("Path to dataset files:", path)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


100%|██████████| 193M/193M [00:07<00:00, 26.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/berkanoztas/synthetic-transaction-monitoring-dataset-aml/versions/2


In [ ]:
data=pd.read_csv("/root/.cache/kagglehub/datasets/berkanoztas/synthetic-transaction-monitoring-dataset-aml/versions/2/SAML-D.csv")


In [ ]:
import numpy as np
import pandas as pd
import networkx as nx


data['Date'] = pd.to_datetime(data['Date'])
data['Time'] = pd.to_timedelta(data['Time'])



def sample_data(df, validation_dt=70, test_dt=40):
  # validation_cut_off=70: selects transactions for validation set, from the final 70 days prior to the dataset's latest date
  # test_cut_off=40: selects transactions for test set, from the final 70 days prior to the dataset's latest date

  test_cutoff = df['Date'].max() - pd.Timedelta(days=test_dt)
  validation_cutoff = df['Date'].max() - pd.Timedelta(days=validation_dt)

  test_size = len(df[df.Date >= test_cutoff])
  validation_size = len(df[(df.Date >= validation_cutoff) & (df.Date < test_cutoff)])
  train_size = len(df[df.Date < validation_cutoff])

  test_percentage = test_size/len(df)
  validation_percentage = validation_size/len(df)
  train_percentage = train_size/len(df)

  print(f"Test size: {test_size} ({test_percentage*100:.2f}%)")
  print(f"Validation size: {validation_size} ({validation_percentage*100:.2f}%)")
  print(f"Train size: {train_size} ({train_percentage*100:.2f}%)")

  test_laundering = df[df.Date >= test_cutoff].Is_laundering.sum()/len(df[df.Date >= test_cutoff])
  validation_laundering = df[(df.Date >= validation_cutoff) & (df.Date < test_cutoff)].Is_laundering.sum()/len(df[(df.Date >= validation_cutoff) & (df.Date < test_cutoff)])
  train_laundering = df[df.Date < validation_cutoff].Is_laundering.sum()/len(df[df.Date < validation_cutoff])

  train = df[df.Date < validation_cutoff]
  val = df[(df.Date >= validation_cutoff) & (df.Date < test_cutoff)]
  test = df[df.Date >= test_cutoff]

  train.to_csv("train.csv")
  val.to_csv("validation.csv")
  test.to_csv("test.csv")

  print(f"Laundering proportion (Test set): {test_laundering}")
  print(f"Laundering proportion (Validation set): {validation_laundering}")
  print(f"Laundering proportion (Train set): {train_laundering}")

In [ ]:
sample_data(data, 70, 35)

Test size: 1030388 (10.84%)
Validation size: 1044845 (10.99%)
Train size: 7429619 (78.17%)
Laundering proportion (Test set): 0.0011626688199008529
Laundering proportion (Validation set): 0.0011446673908570171
Laundering proportion (Train set): 0.0010066465050226666
